In [ ]:
# =========================================================
# SEMMA: Sample, Explore, Modify, Model, Assess
# Dataset: UCI Student Performance (Portuguese course)
# Goal: Predict final grade (G3)
# =========================================================

import os, urllib.request, zipfile, io, warnings
warnings.filterwarnings("ignore")

import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# === SETUP ===
os.makedirs("artifacts", exist_ok=True)
os.makedirs("data", exist_ok=True)
os.makedirs("artifacts/figs", exist_ok=True)

# =========================================================
# === SAMPLE PHASE ===
# =========================================================
# Obtain UCI Student Performance dataset (Portuguese course)
URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip"
z = io.BytesIO(urllib.request.urlopen(URL).read())
zipfile.ZipFile(z).extractall("data")
df = pd.read_csv("data/student-mat.csv", sep=";")

# Define target and predictors
target = "G3"
y = df[target].values
X = df.drop(columns=[target])

print("Data shape:", df.shape)
print("Target distribution summary:")
print(df[target].describe())

# =========================================================
# === EXPLORE PHASE ===
# =========================================================
# Visualize correlations
corr = pd.get_dummies(df).corr()["G3"].sort_values(ascending=False)
print("\nTop correlations with G3:")
print(corr.head(10))

plt.figure(figsize=(6,5))
sns.heatmap(pd.get_dummies(df).corr()[["G3"]].sort_values("G3", ascending=False).head(20),
            annot=False, cmap="coolwarm")
plt.title("Top correlations with G3")
plt.savefig("artifacts/figs/corr_semma.png")
plt.close()

# =========================================================
# === MODIFY PHASE ===
# =========================================================
# Feature engineering
X["failures_bin"] = (df["failures"] > 0).astype(int)
X["studytime_cat"] = df["studytime"].astype("category")

# Define numeric and categorical feature groups
num_cols = X.select_dtypes(include="number").columns.tolist()
cat_cols = X.select_dtypes(exclude="number").columns.tolist()

# Transformation pipelines
num_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc", StandardScaler())
])
cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("oh", OneHotEncoder(handle_unknown="ignore"))
])

pre = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])

# =========================================================
# === MODEL PHASE ===
# =========================================================
models = {
    "rf": RandomForestRegressor(n_estimators=400, random_state=42),
    "xgb": XGBRegressor(
        n_estimators=400, learning_rate=0.05, max_depth=5,
        subsample=0.9, colsample_bytree=0.9, random_state=42
    )
}

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

best_name, best_pipe, best_score = None, None, -999
# StratifiedKFold based on grade bins for balanced folds
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42).split(
    pd.cut(y_train, bins=5, labels=False), np.zeros_like(y_train)
)

for name, est in models.items():
    pipe = Pipeline([("pre", pre), ("model", est)])
    scores = []
    for tr_idx, va_idx in cv:
        pipe.fit(X_train.iloc[tr_idx], y_train[tr_idx])
        pred = pipe.predict(X_train.iloc[va_idx])
        scores.append(r2_score(y_train[va_idx], pred))
    mean_r2 = np.mean(scores)
    print(f"{name} mean CV R2: {mean_r2:.4f}")
    if mean_r2 > best_score:
        best_name, best_pipe, best_score = name, pipe, mean_r2

print("\nBest model:", best_name, "| Mean CV R2:", round(best_score, 4))

# =========================================================
# === ASSESS PHASE ===
# =========================================================
best_pipe.fit(X_train, y_train)
pred = best_pipe.predict(X_test)
r2 = r2_score(y_test, pred)
mae = mean_absolute_error(y_test, pred)

print("\n=== FINAL MODEL PERFORMANCE ===")
print(f"Selected model: {best_name}")
print(f"R²: {r2:.4f} | MAE: {mae:.4f}")
if best_name == "xgb":
    print("Interpretation: XGBoost captured nonlinear grade predictors (failures, studytime, absences).")
else:
    print("Interpretation: RandomForest offered stable generalization and feature robustness.")

# Save artifacts
import joblib, json
joblib.dump(best_pipe, "artifacts/model_reg.pkl")
with open("artifacts/metrics.json", "w") as f:
    json.dump({"r2": float(r2), "mae": float(mae), "model": best_name}, f, indent=2)
print("\nArtifacts saved successfully to artifacts/ folder.")

best_name, best_score


Data shape: (395, 33)
Target distribution summary:
count    395.000000
mean      10.415190
std        4.581443
min        0.000000
25%        8.000000
50%       11.000000
75%       14.000000
max       20.000000
Name: G3, dtype: float64

Top correlations with G3:
G3             1.000000
G2             0.904868
G1             0.801468
Medu           0.217147
higher_yes     0.182465
Fedu           0.152457
romantic_no    0.129970
Mjob_health    0.116158
address_U      0.105756
sex_M          0.103456
Name: G3, dtype: float64
rf mean CV R2: 0.8843
xgb mean CV R2: nan

Best model: rf | Mean CV R2: 0.8843

=== FINAL MODEL PERFORMANCE ===
Selected model: rf
R²: 0.8058 | MAE: 1.1942
Interpretation: RandomForest offered stable generalization and feature robustness.

Artifacts saved successfully to artifacts/ folder.


('rf', np.float64(0.8843214956669773))